In [ ]:
### Step 1: Emotional Check-in:

# Input: User describes their feelings or workday.
# LLM Task: Analyze the input for emotional tone and identify keywords (e.g., "stress," "boredom").
# Output: A summary of emotional trends.


In [ ]:
print("I am your well being companion and end goal is to help you in your career.\nI want to start by asking about your feelings, how was your day today.\n")
print("I will do my best as well being companion to analyze your day and come up with the suggestions that might help you in your career and life. \n")

In [ ]:
how_was_day = ""

In [ ]:
how_was_day = input("How was your day today,can you describe about your day, what went well, what did not go well, what you did not like :\n")

In [ ]:
what_went_well = input("What went well for you , today?")

In [ ]:
what_went_bad = input("What did not go well, today?")

In [ ]:
how_was_day = how_was_day + what_went_well + what_went_bad
print(how_was_day)

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import json
from IPython.display import Markdown, display, update_display

In [ ]:
#Initialize environment and constants:
load_dotenv(override=True)

api_key = os.getenv('OPENAI_API_KEY')
if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [1]:
### Step 2: From day spent and what went good, what went bad  => LLM will extract feelings, emotions from those unspoken words :)

In [ ]:
system_prompt_for_emotion_check_in = "You are a career well-being assistant. Your task is to analyze the user's emotional state based on their text input."\
"Look for signs of stress, burnout, dissatisfaction, boredom, motivation, or any other emotional indicators related to work."\
"Based on the input, provide a summary of the user's feelings and categorize them under relevant emotional states (e.g., ‘Burnout,’ ‘Boredom,’ ‘Stress,’ ‘Satisfaction,’ etc.)."\
"Your response should be empathetic and non-judgmental. Please summarize the list of feelings, emotions , those unspoken but unheard feelings you get it.\n"

In [ ]:
def get_feelings(how_was_day):
    response = openai.chat.completions.create(
        model=MODEL,
        messages = [
            {'role':'system','content': system_prompt_for_emotion_check_in},
            {'role':'user', 'content': how_was_day}
        ]
    )
    result = response.choices[0].message.content
    return result

In [ ]:
## LLM will give the feelings you have based on "the day you had today".
print(get_feelings(how_was_day))


In [ ]:
### Step 3: From those feelings, emotions ==> Get suggestions from LLM.

In [ ]:
## Lets construct system prompt for LLM to get suggestions (from these feelings above).

system_prompt_for_suggestion ="You are a career well-being assistant.Provide a list of practical,actionable suggestions to help them improve their emotional state."

system_prompt_for_suggestion+="The suggestions should be personalized based on their current feelings, and they should be simple, effective actions the user can take immediately."\
"Include activities, tasks, habits, or approaches that will either alleviate stress, boost motivation, or help them reconnect with their work in a positive way."\
"Be empathetic, non-judgmental, and encouraging in your tone.\n"
system_prompt_for_suggestion += "Request you to respond in JSON format. Below is example:\n"
system_prompt_for_suggestion += '''
{
  "suggestions": [
    {
      "action": "Take a short break",
      "description": "Step away from your workspace for 5-10 minutes. Use this time to take deep breaths, stretch, or grab a drink. This mini-break can help clear your mind and reduce feelings of overwhelm."
    },
    {
      "action": "Write a quick journal entry",
      "description": "Spend 5-10 minutes writing down your thoughts and feelings. Specify what's distracting you and what you appreciate about your personal life. This can help you process emotions and refocus on tasks."
    },
    {
      "action": "Set a small task goal",
      "description": "Choose one manageable task to complete today. Break it down into smaller steps to make it less daunting. Completing even a small task can give you a sense of achievement and boost motivation."
    }
    ]
}
'''


In [ ]:
##  Lets build user prompt to ask LLM for the suggestions based on the feelings above.
## Note: Here while building user_prompt, we are making another LLM call (via function get_feelings() to get feelings analyzed from "day spent".
## Because first step is to get feelings from day spent then we move to offer suggestions to ease discomfort feelings.

def get_user_prompt_for_suggestion(how_was_day):
    user_prompt_for_suggestion = "You are a career well-being assistant.Please see below user’s emotional input on 'day user had spent' and this user input might have feeling burnt out, bored, uninspired, or stressed or sometime opposite "\
    "of these feelings."
    user_prompt_for_suggestion += f"{get_feelings(how_was_day)}"
    return user_prompt_for_suggestion
    

In [ ]:
print(get_user_prompt_for_suggestion(how_was_day))

In [ ]:
def get_suggestions(how_was_day):
    response = openai.chat.completions.create(
        model=MODEL,
        messages = [
            {'role': 'system', 'content':system_prompt_for_suggestion},
            {'role': 'user', 'content': get_user_prompt_for_suggestion(how_was_day)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)
    #display(Markdown(result))

In [ ]:
suggestions = get_suggestions(how_was_day)
print(suggestions)

In [ ]:
### Step 4: From those suggestions from companion ==> Enhance with support you need to follow sugestions like action plan for your self.

In [ ]:
system_prompt_for_enhanced_suggestions = "You are a helpful assistant that enhances actionable suggestions for users. For each suggestion provided, enhance it by adding:\n"\
"1. A step-by-step guide for implementation."\
"2. Tools, resources, or apps that can help."\
"3. Examples or additional context to make the suggestion practical."


In [ ]:
def get_user_prompt_for_enhanced_suggestions(suggestions):
     prompt = "You are able to check below suggestions and can enhance to help end user. Below is the list of suggestions.\n"
     prompt += f"{suggestions}"
     return prompt

In [ ]:
def enhance_suggestions(suggestions):
    stream = openai.chat.completions.create(
        model = MODEL,
        messages=[
            {'role':'system', 'content':system_prompt_for_enhanced_suggestions},
            {'role':'user', 'content':get_user_prompt_for_enhanced_suggestions(suggestions)}
        ],
        stream = True
    )
    
    #result = response.choices[0].message.content
    #for chunk in stream:
    #    print(chunk.choices[0].delta.content or '', end='')

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)
    
    #display(Markdown(result))


In [ ]:
detailed = input("\nWould you like a DETAILED PLAN for implementing this suggestion?(Yes/ No)")

In [ ]:
if detailed.lower() == 'yes':
    enhance_suggestions(suggestions)
else:
    print(suggestions)
    